# Universe creation and manipulation

In [1]:
import MDAnalysis as mda
import nglview as nv

_ColormakerRegistry()

* There is more than one way of creating a `Universe` depending on the needs;
* The content of a `Universe` can be altered;
* MDAnalysis can talk with other libraries.

## Creating a `Universe`

### The boring way

In [3]:
import MDAnalysis as mda
from MDAnalysis.tests.datafiles import TPR, XTC
u = mda.Universe(TPR, XTC)

```python
mda.Universe(topology, trajectory)
```

### From a numpy array

### From scratch

### By copying another `Universe`

### By merging atom groups

## Moving atoms

### Setting atom positions

In [ ]:
u.atoms.positions = np.array(...)

### Be careful! `AtomGroup.positions` returns a copy

In [ ]:
print(u.atoms.positions)
u.atoms.positions += 10
print(u.atoms.positions)

### Translation

In [ ]:
u.atoms.positions = u.atoms.positions + ...

In [ ]:
u.atoms.translate(...)

Building a crystal

### Rotation

In [ ]:
u.atoms.rotate(...)


In [ ]:
u.atoms.rotateby(...)

### Geometric transformations

In [ ]:
u.atoms.transform(...)

### Periodic boundary conditions

In [ ]:
u.atoms.unwrap()

In [ ]:
u.atoms.wrap()

In [ ]:
u.atoms.pack_into_box(...)

### Be careful! Changes may not be permanent!

Tomorrow: on-the-fly transformations

## Talking to other libraries

### Using files

### Parmed

### RDKit

### OpenMM